In [1]:
%load_ext Cython
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower', interpolation='nearest')

In [2]:
import os
os.environ['CC'] = 'gcc-6'

In [57]:
%%cython --compile-args=-Ofast

cimport cython
from cython.parallel import prange, parallel

ctypedef double real_t
cdef struct real2_t:
    real_t x, y, z
cdef struct particle_t:
    real_t x, y, vx, vy, vz

cdef class grid_t:
    """Grid extension type.
    This is inherited by the Grid class (see grid.py)."""
    # TODO: Either define *all* attributes of the grid class or only those that
    # are actually needed
    cdef public int noff
    cdef public int lbx, lby

@cython.boundscheck(False)
@cython.wraparound(False)
def deposit_inline(
        particle_t[:] particles, real_t[:, :] density, real2_t[:,:] J, grid_t grid, real_t S):

    cdef int Np = particles.shape[0]
    cdef int ip

    # Density deposition
    for ip in range(Np):
        deposit_particle(particles[ip], density, J, grid, S)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline void deposit_particle(particle_t particle, real_t[:,:] density,
                    real2_t[:,:] J, grid_t grid, real_t S) nogil:

        cdef int ix, iy
        cdef real_t tx, ty, dx, dy

        ix = <int> particle.x
        iy = <int> particle.y

        dx = particle.x - <real_t> ix
        dy = particle.y - <real_t> iy

        tx = 1.0 - dx
        ty = 1.0 - dy

        iy = iy - grid.noff

        ix = ix + grid.lbx
        iy = iy + grid.lby

        density[iy  , ix  ] += ty*tx
        density[iy  , ix+1] += ty*dx
        density[iy+1, ix  ] += dy*tx
        density[iy+1, ix+1] += dy*dx

        J[iy  , ix  ].x += ty*tx*(particle.vx + S*particle.y)
        J[iy  , ix+1].x += ty*dx*(particle.vx + S*particle.y)
        J[iy+1, ix  ].x += dy*tx*(particle.vx + S*particle.y)
        J[iy+1, ix+1].x += dy*dx*(particle.vx + S*particle.y)

        J[iy  , ix  ].y += ty*tx*particle.vy
        J[iy  , ix+1].y += ty*dx*particle.vy
        J[iy+1, ix  ].y += dy*tx*particle.vy
        J[iy+1, ix+1].y += dy*dx*particle.vy

        J[iy  , ix  ].z += ty*tx*particle.vz
        J[iy  , ix+1].z += ty*dx*particle.vz
        J[iy+1, ix  ].z += dy*tx*particle.vz
        J[iy+1, ix+1].z += dy*dx*particle.vz

@cython.boundscheck(False)
@cython.wraparound(False)
def deposit(
        particle_t[:] particles, real_t[:, :] density, real2_t[:,:] J, grid_t grid, real_t S):

    cdef int Np = particles.shape[0]
    cdef int ip
    cdef int ix, iy
    cdef real_t tx, ty, dx, dy

    # Density deposition
    for ip in range(Np):

        ix = <int> particles[ip].x
        iy = <int> particles[ip].y

        dx = particles[ip].x - <real_t> ix
        dy = particles[ip].y - <real_t> iy

        tx = 1.0 - dx
        ty = 1.0 - dy

        iy = iy - grid.noff

        ix = ix + grid.lbx
        iy = iy + grid.lby

        density[iy  , ix  ] += ty*tx
        density[iy  , ix+1] += ty*dx
        density[iy+1, ix  ] += dy*tx
        density[iy+1, ix+1] += dy*dx

        J[iy  , ix  ].x += ty*tx*(particles[ip].vx + S*particles[ip].y)
        J[iy  , ix+1].x += ty*dx*(particles[ip].vx + S*particles[ip].y)
        J[iy+1, ix  ].x += dy*tx*(particles[ip].vx + S*particles[ip].y)
        J[iy+1, ix+1].x += dy*dx*(particles[ip].vx + S*particles[ip].y)

        J[iy  , ix  ].y += ty*tx*particles[ip].vy
        J[iy  , ix+1].y += ty*dx*particles[ip].vy
        J[iy+1, ix  ].y += dy*tx*particles[ip].vy
        J[iy+1, ix+1].y += dy*dx*particles[ip].vy

        J[iy  , ix  ].z += ty*tx*particles[ip].vz
        J[iy  , ix+1].z += ty*dx*particles[ip].vz
        J[iy+1, ix  ].z += dy*tx*particles[ip].vz
        J[iy+1, ix+1].z += dy*dx*particles[ip].vz

In [61]:
class Grid(grid_t):
    def __init__(self, nx, ny):
        self.noff = 0
        self.lbx = 1
        self.lby = 1
        self.nx = nx
        self.ny = ny

nx = 256
ny = 256
grid = Grid(nx, ny)

In [64]:
import numpy as np
Float = np.double
Float2 = [('x', Float), ('y', Float), ('z', Float)]

Particle = np.dtype(
    [('x', Float), ('y', Float), ('vx', Float), ('vy', Float),
    ('vz', Float)], align=True)

npc = 256
N = nx*ny*npc
particles = np.empty(N, dtype=Particle)

In [65]:
sqrt_npc = int(np.sqrt(npc))
assert sqrt_npc**2 == npc
dx = dy = 1/sqrt_npc
x, y = np.meshgrid(
    nx*np.arange(sqrt_npc+0.5)/sqrt_npc,
    ny*np.arange(sqrt_npc+0.5)/sqrt_npc)

x, y = np.meshgrid(
    (np.arange(nx*sqrt_npc) + 0.5)/sqrt_npc,
    (np.arange(ny*sqrt_npc) + 0.5)/sqrt_npc)

particles['x'] = x.flatten()
particles['y'] = y.flatten()

In [66]:
%%timeit -n 10 rho = np.zeros((ny + 2, nx + 2), dtype=Float); J = np.zeros((ny + 2, nx + 2), dtype=Float2);
deposit_inline(particles, rho, J, grid, 0.0)

10 loops, best of 3: 179 ms per loop


In [67]:
%%timeit -n 10 rho = np.zeros((ny + 2, nx + 2), dtype=Float); J = np.zeros((ny + 2, nx + 2), dtype=Float2);
deposit(particles, rho, J, grid, 0.0)

10 loops, best of 3: 178 ms per loop


In [68]:
particles['x'] = np.random.uniform(size=N)*nx
particles['y'] = np.random.uniform(size=N)*ny

In [71]:
%%timeit -n 10 rho = np.zeros((ny + 2, nx + 2), dtype=Float); J = np.zeros((ny + 2, nx + 2), dtype=Float2);
deposit_inline(particles, rho, J, grid, 0.0)

10 loops, best of 3: 285 ms per loop


In [72]:
%%timeit -n 10 rho = np.zeros((ny + 2, nx + 2), dtype=Float); J = np.zeros((ny + 2, nx + 2), dtype=Float2);
deposit(particles, rho, J, grid, 0.0)

10 loops, best of 3: 291 ms per loop
